In [ ]:
import os
import sys
import pandas as pd
from IPython.display import display, HTML

# setting warnings
import warnings
warnings.simplefilter(action='ignore', category = Warning)

# add parent folder path to the namespace
sys.path.append(os.path.dirname(os.getcwd()))

# import modules and components
from utils.data_assets import PreProcessing, DataValidation
import utils.global_paths as globpt
import configurations as cnf

# specify relative paths from global paths and create subfolders
mat_path = os.path.join(globpt.data_path, 'materials') 
exp_path = os.path.join(globpt.data_path, 'experiments') 
os.mkdir(mat_path) if not os.path.exists(mat_path) else None
os.mkdir(exp_path) if not os.path.exists(exp_path) else None 

# Load and prepare data

In [ ]:
filepath = os.path.join(mat_path, 'adsorbents_dataset.csv')  
df_adsorbents = pd.read_csv(filepath, sep=';', encoding='utf-8')  
filepath = os.path.join(mat_path, 'adsorbates_dataset.csv')  
df_adsorbates = pd.read_csv(filepath, sep=';', encoding='utf-8')  
filepath = os.path.join(exp_path, 'single_component_dataset.csv')  
df_SCADS = pd.read_csv(filepath, sep=';', encoding='utf-8')
filepath = os.path.join(exp_path, 'binary_mixture_dataset.csv')  
df_BMADS = pd.read_csv(filepath, sep=';', encoding='utf-8')

# 1. Data analysis

BMADS dataset is not analyzed fully, as it is of less interest. Some general info on BMADS can be found here:

In [ ]:
num_BMADS_experiments = df_BMADS['filename'].nunique()
print('\nBMADS dataset: binary mixture adsorption')
print('-----------------------------------------------')
print(f'Number of measurements: {df_BMADS.shape[0]}')
print(f'Number of actual experiments: {num_BMADS_experiments}')
print(f'Average measurements per experiment: {df_BMADS.shape[0]//num_BMADS_experiments}')

## 1.1 Analyze experiments dataset

SCADS dataset (single component adsorption) is analyzed and validate. The major aim is to provide a fully preprocessed dataset for machine learning. General info on the dataset are provided here:

In [ ]:
preprocessor = PreProcessing()
validator = DataValidation()
valid_units = preprocessor.valid_units

negative_press = df_SCADS[df_SCADS['pressure'] < 0].shape[0]
negative_uptake = df_SCADS[df_SCADS['adsorbed_amount'] < 0].shape[0]
negative_temp = df_SCADS[df_SCADS['temperature'] <= 0].shape[0]
num_experiments = df_SCADS['filename'].nunique()

print('\nSCADS dataset: single component adsorption')
print('-----------------------------------------------')
print(f'Number of measurements: {df_SCADS.shape[0]}')
print(f'Number of actual experiments: {num_experiments}')
print(f'Average measurements per experiment: {df_SCADS.shape[0]//num_experiments}')
print('-----------------------------------------------')
print(f'Negative pressure measurements: {negative_press} ({negative_press/df_SCADS.shape[0] * 100:.2f}%)')
print(f'Negative uptake measurements: {negative_uptake} ({negative_uptake/df_SCADS.shape[0] * 100:.2f}%)')
print(f'Negative temperature measurements: {negative_uptake} ({negative_uptake/df_SCADS.shape[0] * 100:.2f}%)')
print('-----------------------------------------------')
print(f'Number of null values in dataset: \n{df_SCADS.isnull().sum()}')


Distribution of experiments by units

In [ ]:
unique_P_units = df_SCADS['pressureUnits'].unique()
num_unique_P_units = df_SCADS['pressureUnits'].nunique()
unique_Q_units = df_SCADS['adsorptionUnits'].unique()
num_unique_Q_units = df_SCADS['adsorptionUnits'].nunique()

print(f'Number of unique pressure units: {num_unique_P_units}')
print(f'Number of unique uptake units: {num_unique_Q_units}')
print(f'List of valid uptake units: \n{sorted([str(x) for x in valid_units])}\n')

# filter experiments leaving only valid uptake and pressure units, then convert 
# pressure and uptake to Pa (pressure) and mol/kg (uptake)
# filter experiments by pressure and uptake units 
dataset = df_SCADS[df_SCADS[preprocessor.Q_unit_col].isin(preprocessor.valid_units)]
print(f'Number of uptake valid units is {len(valid_units)} vs invalid units {num_unique_Q_units-len(valid_units)}')
print(f'Number of measurements removed by sorting units: {df_SCADS.shape[0] - dataset.shape[0]}')
print(f'Number of remaining measurements upon sorting units: {dataset.shape[0]}')

Visualize distribution of adsorption units per measurements, with all units and only with valid units

In [ ]:
validator = DataValidation()
print('Dataset that has not undergone preprocessing\n')
validator.class_distribution(df_SCADS, 'adsorptionUnits', 'Distribution of experiments per adsorption units')
print('Dataset that has been preprocessed')
validator.class_distribution(dataset, 'adsorptionUnits', 'Distribution of experiments per valid adsorption units')

## 1.2 Explore materials datasets

In [ ]:
print(f'\nNumber of adsorbents from adsorbent dataset: {df_adsorbents['name'].nunique()}')
print(f'Number of adsorbates from adsorbates dataset: {df_adsorbates['name'].nunique()}')